In [ ]:
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix, precision_score
from sklearn.linear_model import LogisticRegression, LinearRegression
from sklearn import svm
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.neural_network import MLPClassifier, MLPRegressor

np.set_printoptions(precision=4)

In [ ]:
data = pd.read_csv('final_df.csv')

In [ ]:
data.head()

## Classification

In [ ]:
df = data.copy()
df.podium = df.podium.map(lambda x: 1 if x == 1 else 0)

train = df[df.season <2019]
X_train = train.drop(['driver', 'podium'], axis = 1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)


In [ ]:
def score_classification(model):
    score = 0
    for circuit in df[df.season == 2019]['round'].unique():

        test = df[(df.season == 2019) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict_proba(X_test), columns = ['proba_0', 'proba_1'])
        prediction_df['actual'] = y_test.reset_index(drop = True)
        prediction_df.sort_values('proba_1', ascending = False, inplace = True)
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / df[df.season == 2019]['round'].unique().max()
    return model_score

In [ ]:
comparison_dict ={'model':[],
                  'params': [],
                  'score': []}

### Logistic Regression

In [ ]:
# Logistic Regression

params={'penalty': ['l1', 'l2'],
        'solver': ['saga', 'liblinear'],
        'C': np.logspace(-3,1,20)}

for penalty in params['penalty']:
    for solver in params['solver']:
        for c in params['C']:
            model_params = (penalty, solver, c)
            model = LogisticRegression(penalty = penalty, solver = solver, C = c, max_iter = 10000)
            model.fit(X_train, y_train)
            
            model_score = score_classification(model)
            
            comparison_dict['model'].append('logistic_regression')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)


### Random Forest Classifier

In [ ]:
# Random Forest Classifier

params={'criterion': ['gini', 'entropy'],
        'max_features': [0.8, 'auto', None],
        'max_depth': list(np.linspace(5, 55, 26)) + [None]}

for criterion in params['criterion']:
    for max_features in params['max_features']:
        for max_depth in params['max_depth']:
            model_params = (criterion, max_features, max_depth)
            model = RandomForestClassifier(criterion = criterion, max_features = max_features, max_depth = max_depth)
            model.fit(X_train, y_train)
            
            model_score = score_classification(model)
            
            comparison_dict['model'].append('random_forest_classifier')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)


In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestClassifier


param_dist = {
    'criterion': ['gini', 'entropy'],
    'max_features': [0.8, 'auto', None],
    'max_depth': list(range(5, 56, 5)) + [None]
}

model = RandomForestClassifier(random_state=1)

random_search = RandomizedSearchCV(model, param_dist, n_iter=10, cv=5, random_state=1)
random_search.fit(X_train, y_train)


best_model = random_search.best_estimator_
best_params = random_search.best_params_
best_score = random_search.best_score_


comparison_dict['model'].append('random_forest_classifier')
comparison_dict['params'].append(best_params)
comparison_dict['score'].append(best_score)


### SVM Classifier

In [ ]:
from sklearn import svm
from sklearn.model_selection import RandomizedSearchCV
import numpy as np

params = {
    'gamma': np.logspace(-4, -1, 20),
    'C': np.logspace(-2, 1, 20),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}

model = svm.SVC(probability=True)
random_search = RandomizedSearchCV(model, params, n_iter=10, cv=5, random_state=1)
random_search.fit(X_train, y_train)

best_model = random_search.best_estimator_
best_params = random_search.best_params_
best_score = random_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

### Neural Network Classifier

In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import RandomizedSearchCV
import numpy as np


params = {
    'hidden_layer_sizes': [(80, 20, 40, 5), (75, 25, 50, 10)],
    'activation': ['identity', 'logistic', 'tanh', 'relu'],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'alpha': np.logspace(-4, 2, 20)
}


model = MLPClassifier(random_state=1, max_iter=500)


random_search = RandomizedSearchCV(model, params, n_iter=10, cv=5, random_state=1)
random_search.fit(X_train, y_train)


best_model = random_search.best_estimator_
best_params = random_search.best_params_
best_score = random_search.best_score_


comparison_dict['model'].append('neural_network_classifier')
comparison_dict['params'].append(best_params)
comparison_dict['score'].append(best_score)


In [ ]:
# Neural network

params={'hidden_layer_sizes': [(80,20,40,5), (75,25,50,10)], 
        'activation': ['identity', 'tanh', 'relu'], 
        'solver': ['lbfgs', 'sgd', 'adam'], 
        'alpha': np.logspace(-4,2,20)} 


for hidden_layer_sizes in params['hidden_layer_sizes']:
    for activation in params['activation']:
        for solver in params['solver']:
            for alpha in params['alpha']:
                model_params = (hidden_layer_sizes, activation, solver, alpha )
                model = MLPClassifier(hidden_layer_sizes = hidden_layer_sizes,
                                      activation = activation, solver = solver, alpha = alpha, random_state = 1)
                model.fit(X_train, y_train)

                model_score = score_classification(model)

                comparison_dict['model'].append('neural_network_classifier')
                comparison_dict['params'].append(model_params)
                comparison_dict['score'].append(model_score)


## Regression

In [ ]:
df = data.copy()

train = df[df.season <2019]
X_train = train.drop(['driver', 'podium'], axis = 1)
y_train = train.podium

scaler = StandardScaler()
X_train = pd.DataFrame(scaler.fit_transform(X_train), columns = X_train.columns)


In [ ]:
def score_regression(model):
    score = 0
    for circuit in df[df.season == 2019]['round'].unique():

        test = df[(df.season == 2019) & (df['round'] == circuit)]
        X_test = test.drop(['driver', 'podium'], axis = 1)
        y_test = test.podium

        #scaling
        X_test = pd.DataFrame(scaler.transform(X_test), columns = X_test.columns)

        # make predictions
        prediction_df = pd.DataFrame(model.predict(X_test), columns = ['results'])
        prediction_df['podium'] = y_test.reset_index(drop = True)
        prediction_df['actual'] = prediction_df.podium.map(lambda x: 1 if x == 1 else 0)
        prediction_df.sort_values('results', ascending = True, inplace = True)
        prediction_df.reset_index(inplace = True, drop = True)
        prediction_df['predicted'] = prediction_df.index
        prediction_df['predicted'] = prediction_df.predicted.map(lambda x: 1 if x == 0 else 0)

        score += precision_score(prediction_df.actual, prediction_df.predicted)

    model_score = score / df[df.season == 2019]['round'].unique().max()
    return model_score

### Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

# Linear Regression
params = {'fit_intercept': [True, False]}

for fit_intercept in params['fit_intercept']:
    model_params = {'fit_intercept': fit_intercept}
    model = LinearRegression(fit_intercept=fit_intercept)
    model.fit(X_train, y_train)
            
    model_score = score_regression(model)
            
    comparison_dict['model'].append('linear_regression')
    comparison_dict['params'].append(model_params)
    comparison_dict['score'].append(model_score)


### Random Forest Regressor

In [ ]:
# Random Forest Regressor

params={'criterion': ['squared_error'], 
        'max_features': [0.8, None], 
        'max_depth': list(range(5, 56, 5)) + [None]}

for criterion in params['criterion']:
    for max_features in params['max_features']:
        for max_depth in params['max_depth']:
            model_params = (criterion, max_features, max_depth)
            model = RandomForestRegressor(criterion=criterion,
                                          max_features=max_features, max_depth=max_depth, random_state=1)
            model.fit(X_train, y_train)
            
            model_score = score_regression(model)
            
            comparison_dict['model'].append('random_forest_regressor')
            comparison_dict['params'].append(model_params)
            comparison_dict['score'].append(model_score)


### SVM Regressor

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVR
import numpy as np


param_dist = {
    'gamma': np.logspace(-4, -1, 10),
    'C': np.logspace(-2, 1, 10),
    'kernel': ['linear', 'poly', 'rbf', 'sigmoid']
}


model = SVR()


random_search = RandomizedSearchCV(model, param_dist, n_iter=10, cv=5, random_state=1)
random_search.fit(X_train, y_train)


best_model = random_search.best_estimator_
best_params = random_search.best_params_
best_score = random_search.best_score_

comparison_dict['model'].append('svm_regressor')
comparison_dict['params'].append(best_params)
comparison_dict['score'].append(best_score)


### Neural Network Regressor

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neural_network import MLPRegressor
import numpy as np


param_dist = {
    'hidden_layer_sizes': [(80,20,40,5), (75,30,50,10,3)],
    'activation': ['identity', 'relu','logistic', 'tanh'],
    'solver': ['lbfgs','sgd', 'adam'],
    'alpha': np.logspace(-4,1,20)
}

model = MLPRegressor(random_state=1)


random_search = RandomizedSearchCV(model, param_dist, n_iter=10, cv=5, random_state=1)
random_search.fit(X_train, y_train)


best_model = random_search.best_estimator_
best_params = random_search.best_params_ b
best_score = random_search.best_score_


comparison_dict['model'].append('nn_regressor')
comparison_dict['params'].append(best_params)
comparison_dict['score'].append(best_score)


In [ ]:
import pandas as pd
pd.DataFrame(comparison_dict).groupby('model')['score'].max()

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt


comparison_df = pd.DataFrame(comparison_dict)
max_scores = comparison_df.groupby('model')['score'].max()

plt.clf()

max_scores.plot(kind='bar', figsize=(10, 6))
plt.title('Maximum Score for Each Model')
plt.xlabel('Model')
plt.ylabel('Max Score')
plt.xticks(rotation=45)
plt.grid(axis='y')


for i, (model, score) in enumerate(max_scores.items()):
    score_text = f'{score:.1f}'
    percent_text = f'{score * 100:.2f}%'
    plt.text(i, score + 0.1, score_text, ha='center', va='bottom')
    plt.text(i, score - 0.1, percent_text, ha='center', va='top')

plt.show()
